<a href="https://colab.research.google.com/github/thecodinguru/DS-Unit-2-Kaggle-Challenge/blob/master/Copy_of_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [ ]:
test.shape

(14358, 40)

In [ ]:
train.shape

(59400, 41)

In [ ]:
import numpy as np

#this function will clean up data. droping features with high cardinality and
#replacing lat and log values with 0s to nan.then it splits the data
def wrangle_split(train, test):

  train_c = train.copy()
  test_c = test.copy()

  #change bad latitute measurements with 0
  train_c['latitude'] = train_c['latitude'].replace(-2e-08, 0)
  test_c['latitude'] = test_c['latitude'].replace(-2e-08, 0)
  
  #remove outliers
  train_c = train_c[(train_c['latitude'] >= np.percentile(train_c['latitude'], 0.05)) & 
        (train_c['latitude'] < np.percentile(train_c['latitude'], 99.95)) &
        (train_c['longitude'] >= np.percentile(train_c['longitude'], 0.05)) & 
        (train_c['longitude'] <= np.percentile(train_c['longitude'], 99.95))]
  #test_c = test_c[(test_c['latitude'] >= np.percentile(test_c['latitude'], 0.05)) & 
        #(test_c['latitude'] < np.percentile(test_c['latitude'], 99.95)) &
        #(test_c['longitude'] >= np.percentile(test_c['longitude'], 0.05)) & 
        #(test_c['longitude'] <= np.percentile(test_c['longitude'], 99.95))]

  #change all the latitude and longitude values that are 0 with nan
  cols_with_zeros = ['longitude', 'latitude']
  for col in cols_with_zeros:
     train_c[col] = train_c[col].replace(0, np.nan)
     test_c[col] = test_c[col].replace(0, np.nan)

  #get rid of features with high cardinality
  hc_cols_train = [col for col in train_c.describe(include='object').columns 
             if train_c[col].nunique() > 100]
  hc_cols_test = [col for col in test_c.describe(include='object').columns 
             if test_c[col].nunique() > 100]


  #Droping a repeated feature
  train_c = train_c.drop(['quantity_group'] + hc_cols_train, axis=1)
  test_c = test_c.drop(['quantity_group'] + hc_cols_test , axis=1)
  
  y = train_c['status_group']
  X = train_c.drop('status_group', axis=1)

  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=77, stratify=y)

  return X_train, X_val, y_train, y_val, test_c

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#applying the wrange split function to the data
X_train, X_val, y_train, y_val, test_c = wrangle_split(train, test)

#baseline

In [ ]:
#baseline accuracy
baseline = y_train.value_counts(normalize=True).max()
baseline

0.5450871322411021

#model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
model = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(random_state=50)
)

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'recorded_by', 'scheme_management',
                                      'permit', 'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'w...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                

In [ ]:
train_accuracy = model.score(X_train, y_train)
val_accuracy = model.score(X_val, y_val)

print(train_accuracy)
print(val_accuracy)

0.9999782712615706
0.8082739440292022


#tuning

In [ ]:
model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=50)
)

params = {'randomforestclassifier__n_estimators': range(1, 200, 10),
          'randomforestclassifier__max_depth': range(1,20)}

rs = RandomizedSearchCV(model, param_distributions=params, 
                        n_iter=10, 
                        scoring='accuracy',
                        n_jobs=-1,
                        verbose=1,
                        cv=3)

rs.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.8min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [ ]:
best_model = rs.best_estimator_

In [ ]:
train_accuracy = best_model.score(X_train, y_train)
val_accuracy = best_model.score(X_val, y_val)

print(train_accuracy)
print(val_accuracy)

0.9438094824214506
0.8054058751955502


In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
submission4 = sample_submission.copy()
y_predit_log = best_model.predict(test_c)
submission4['status_group'] = y_predit_log
submission4.to_csv('uzomeziem_eze_kaggle.csv', index=False)

from google.colab import files
files.download('uzomeziem_eze_kaggle.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>